### Respostas

**RESOLVER PROBLEMA DA VARIÁVEL DE AMBIENTE**

#### 1. Identificação e Escolha do Modelo LLM (Local):

| Modelo                             | Desempenho                              | Custo Computacional Local              | Acessibilidade Local |
|------------------------------------|-----------------------------------------|----------------------------------------|----------------------|
| gpt2                               | 2.6s mas não respondeu adequadamente    | Custo mínimo                           | Acessível            |
| bigscience/bloom-560m              | 12s resposta estranha, mas não alucinou | Custo mínimo                           | Acessível            |
| TinyLlama/TinyLlama-1.1B-Chat-v1.0 | 3min e 11 com uma resposta adequada     | Necessidade de mais de 8GB de memória  | Acessível            |
| gpt2-large                         | 8,6 mas não respondeu adequadamente     | Custo mínimo                           | Acessível            |
| EleutherAI/gpt-neo-1.3B            | Não rodou localmente                    | Necessidade de mais de 24GB de memória | Acessível            |
| tiiuae/falcon-7b                   | Não rodou localmente                    | Necessidade de mais de 12GB de memória | Acessível            |

**R.:** Considerando apenas as opções que rodaram localmente e a qualidade das respostas, o TinyLlama-1.1B foi escolhido, pois, apesar demorar mais para responder, a sua resposta foi mais assertiva, compensando seu maior custo computacional para ser usada.

#### 2. Integração de LLM com FastAPI no Ambiente Local:

uvicorn main:api --reload --port 8000

In [1]:
import requests
import pandas as pd
import json

def get_stocks_valorization(ticker):
    url = 'http://127.0.0.1:8000/stock/valorization'
    response = requests.get(url, params={'ticker': ticker})
    data = response.json()
    return data

def get_dollar_valorization():
    url = 'http://127.0.0.1:8000/indicators/dollar/valorization/'
    response = requests.get(url,)
    data = response.json()
    return data

def get_response(msg : str, data):
    url = 'http://127.0.0.1:8000/llm/question'
    response = requests.get(url,params={'question':msg, 'data': data})
    return response

In [2]:
#MENSAGEM PARA A API
msg = 'This stock is a the good investment?'

In [ ]:
sJson = json.loads(get_stocks_valorization('PETR4.SA'))
djson = json.loads(get_dollar_valorization())

#Foi necessário reduzir o tamanho do json para 6 meses para não exceder o limite de tokens
#Reduzido para 1 mês para respostas mais rápidas
stock_last_6_months = pd.DataFrame(sJson).astype(str).tail(1).to_dict(orient='list')
dollar_last_6_months = pd.DataFrame(djson).astype(str).tail(1).to_dict(orient='list')

#Tempo médio de resposta: 8 a 10 mins com 6 meses
#Tempo médio de resposta: 2 a 3 min com 1 mês
response = get_response(msg, {'stock': stock_last_6_months, 'dollar': dollar_last_6_months})

In [11]:
response.json()

{'answer': 'This stock is a good investment',
 'confidence': 90,
 'justification': 'The company has a strong financial position, a solid management team, and a proven track record of growth and profitability. Additionally, the stock has been trading at a discount to its peers, making it an attractive investment opportunity.'}

### Fora da API

In [41]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed
import torch

def llama(question: str, data):
    device = torch.device("cpu")

    pipe = pipeline("text-generation",
                    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                    torch_dtype=torch.bfloat16,
                    device=device)
    message = [
        {"role": "user",
            "content": "You are a investiment advisor, and you have to answer the following question:"},
        {"role": "user",
            "content": question},
        {"role": "user",
            "content": f'''Answer the question above using the data below: {data}'''},
        {"role": "user",
            "content": 
            '''You must return the answer in json format, with the following structure: 
            {
                'answer': 'your answer here',
                'confidence': 'your confidence here',
                'justification': 'your short justification here',
            }
            '''}
        ]

    prompt = pipe.tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=False
    )

    prediction = pipe(prompt,
                    max_new_tokens=1000,
                    do_sample=True,
                    temperature=0.2, top_k=50, top_p=0.95)

    generated_text = prediction[0]['generated_text']


    response = generated_text[len(prompt):].strip()
    return response

Tempo médio para resposta: 8 min

In [93]:
sJson = json.loads(get_stocks_valorization('PETR4.SA'))

dollar_json = json.loads(get_dollar_valorization())

notebook_response = llama(msg, {'stock': pd.DataFrame(sJson).astype(str).tail(1).to_dict(orient='list'), 'dollar': pd.DataFrame(dollar_json).astype(str).tail(1).to_dict(orient='list')})

print(notebook_response)

<|assistant|>
Here's an example of how to return the answer and confidence in JSON format:

```json
{
    "answer": "This stock is a good investment.",
    "confidence": 90,
    "justification": "The data provided in the question and the stock data show that this stock is valued at $5.6615 and has a year-month valuation of 2024-07. The valuation is higher than the current market price of $5.6615, indicating that this stock is a good investment opportunity."
}
```


In [95]:
Notebook_response_clean = json.loads('{' + notebook_response.split("{")[1].split("}")[0] +'}')
Notebook_response_clean

{'answer': 'This stock is a good investment.',
 'confidence': 90,
 'justification': 'The data provided in the question and the stock data show that this stock is valued at $5.6615 and has a year-month valuation of 2024-07. The valuation is higher than the current market price of $5.6615, indicating that this stock is a good investment opportunity.'}